# Optimization Model to Markdown Strategy

## Import data and pre-processing

In [19]:
import pandas as pd
import numpy as np

# Load the Excel file
data = pd.read_excel('scraping_results.xlsx')

data.head(20)

,Unnamed: 0,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect
0,0,1,60,87,1913,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892
1,1,2,60,74,1839,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892
2,2,3,54,147,1692,"Your revenue: $85,752, Perfect foresight strat...",1,15.8,85752,101892
3,3,4,54,45,1647,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892
4,4,5,48,160,1487,"Your revenue: $85,752, Perfect foresight strat...",2,15.8,85752,101892
5,5,6,48,239,1248,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892
6,6,7,48,137,1111,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892
7,7,8,36,261,850,"Your revenue: $85,752, Perfect foresight strat...",3,15.8,85752,101892
8,8,9,36,292,558,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892
9,9,10,36,294,264,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892


In [3]:
# Assuming each item's data is in blocks of 15 rows. Extract data for each item.
item_blocks = [data.iloc[i*15:(i+1)*15] for i in range(len(data) // 15)]
demand_lifts = []

In [9]:
# print sample item_block
item_blocks[1]

,Unnamed: 0,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect
15,15,1,60,79,1921,"Your revenue: $79,188, Perfect foresight strat...",0,9.0,79188,87054
16,16,2,60,59,1862,"Your revenue: $79,188, Perfect foresight strat...",0,9.0,79188,87054
17,17,3,54,86,1776,"Your revenue: $79,188, Perfect foresight strat...",1,9.0,79188,87054
18,18,4,54,80,1696,"Your revenue: $79,188, Perfect foresight strat...",0,9.0,79188,87054
19,19,5,48,74,1622,"Your revenue: $79,188, Perfect foresight strat...",2,9.0,79188,87054
20,20,6,36,219,1403,"Your revenue: $79,188, Perfect foresight strat...",3,9.0,79188,87054
21,21,7,36,120,1283,"Your revenue: $79,188, Perfect foresight strat...",0,9.0,79188,87054
22,22,8,36,224,1059,"Your revenue: $79,188, Perfect foresight strat...",0,9.0,79188,87054
23,23,9,36,130,929,"Your revenue: $79,188, Perfect foresight strat...",0,9.0,79188,87054
24,24,10,36,165,764,"Your revenue: $79,188, Perfect foresight strat...",0,9.0,79188,87054


## Calculate Demand Lift

In our dataset, there are 100 different items. Each item has a different demand at different price point. Given that the relative demand lift is the same, we extracted the demand lift for each price point for each item relative to the demand at week 1. Thereafter, we take the average of the demand lift for each price point to get 4 demand lift values.

In [4]:
# Calculate demand lift for each block (representing an item)
for block in item_blocks:
    week_1_sales = block[block['Week'] == 1]['Sales'].values[0]
    demand_lift = block.assign(Demand_Lift=lambda x: x['Sales'] / week_1_sales)
    demand_lifts.append(demand_lift)

# Combine all item blocks back into a single dataframe
demand_lifts_df = pd.concat(demand_lifts)

# Calculate the average demand lift for each price point across all items
average_demand_lifts = demand_lifts_df.groupby('Price')['Demand_Lift'].mean().reset_index()

# Output the results
print(average_demand_lifts)

   Price  Demand_Lift
0     36     1.772344
1     48     1.752019
2     54     1.316609
3     60     0.992055


In [11]:
demand_lifts_df.head(15)

,Unnamed: 0,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect,Demand_Lift
0,0,1,60,87,1913,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,1.000000
1,1,2,60,74,1839,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,0.850575
2,2,3,54,147,1692,"Your revenue: $85,752, Perfect foresight strat...",1,15.8,85752,101892,1.689655
3,3,4,54,45,1647,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,0.517241
4,4,5,48,160,1487,"Your revenue: $85,752, Perfect foresight strat...",2,15.8,85752,101892,1.839080
5,5,6,48,239,1248,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,2.747126
6,6,7,48,137,1111,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,1.574713
7,7,8,36,261,850,"Your revenue: $85,752, Perfect foresight strat...",3,15.8,85752,101892,3.000000
8,8,9,36,292,558,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,3.356322
9,9,10,36,294,264,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,3.379310


## Optimization Problem Formulation

Maximize the revenue:

$$ \\$60 \cdot 0.992055 \cdot d_{1} \cdot x_{60} + \\$54 \cdot 1.316609 \cdot d_{1} \cdot x_{54} + \\$48 \cdot 1.752019 \cdot d_{1} \cdot x_{48} + \\$36 \cdot 1.772344 \cdot d_{1} \cdot x_{36} $$

where:
- $d_{1}$ = initial demand at week 1
- $x_{60}, x_{54}, x_{48}, x_{36}$ = number of weeks the item is sold at \\$60, \\$54, \\$48, and \\$36 respectively

Subject to:
- $ 0.992055 \cdot d_{1} \cdot x_{60} + 1.316609 \cdot d_{1} \cdot x_{54} + 1.752019 \cdot d_{1} \cdot x_{48} + 1.772344 \cdot d_{1} \cdot x_{36}$
- $x_{60} + x_{54} + x_{48} + x_{36} \leq 15$
- $x_{60} \geq 1$
- $x_{54}, x_{48}, x_{36} \geq 0$

## Running the optimization problem using a series of initial demand

Now, we will optimise using the above formulation to determine a suitable markdown strategy for each initial demand, $d_1$, i.e. the demand for a particular item at week 1

In [21]:
# Extract week 1 sales (assuming data is loaded into a DataFrame named 'df')
week_1_sales = data[data['Week'] == 1]['Sales'].values

# Get unique sales values using numpy
unique_sales = np.unique(week_1_sales)

# Print or further use the unique sales values
print(unique_sales)

print(len(unique_sales))

[ 17  25  30  36  44  47  48  49  51  53  54  55  56  61  65  66  67  68
  69  70  71  72  74  76  79  81  83  84  85  86  87  89  90  92  94  96
  98  99 100 101 102 103 104 105 107 108 110 111 112 114 116 117 119 122
 125 127 128 129 133 139 167]
61


Given that there are 61 unique $d_1$ values within the list of 100 items, we will build a series of optimisation model that runs from $d_1$ = 15, 25, 35 ... 175 to recommend the appropriate strategy depending on which buckets the initial $d_1$ falls into.

In [26]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

# Create buckets for week_1_sales
week_1_sales = np.arange(15, 185, 10)

# Placeholder for results
optimization_results = []

for d1 in week_1_sales:
    # Create a new model
    model = gp.Model('PricingStrategy')
    model.setParam('OutputFlag', 0)  # Suppress the optimization log

    # Decision variables
    x60 = model.addVar(lb=1, vtype=GRB.INTEGER, name="x60")
    x54 = model.addVar(lb=0, vtype=GRB.INTEGER, name="x54")
    x48 = model.addVar(lb=0, vtype=GRB.INTEGER, name="x48")
    x36 = model.addVar(lb=0, vtype=GRB.INTEGER, name="x36")

    # Objective function
    model.setObjective(60 * 0.992055 * d1 * x60 +
                       54 * 1.316609 * d1 * x54 +
                       48 * 1.752019 * d1 * x48 +
                       36 * 1.772344 * d1 * x36, GRB.MAXIMIZE)

    # Constraints
    model.addConstr(0.992055 * d1 * x60 + 1.316609 * d1 * x54 + 1.752019 * d1 * x48 + 1.772344 * d1 * x36 <= 2000, "c0")
    model.addConstr(x60 + x54 + x48 + x36 <= 15, "c1")

    # Solve model
    model.optimize()

    # Store results if an optimal solution is found
    if model.status == GRB.OPTIMAL:
        results = {
            'd1': d1,
            'x60': x60.X,
            'x54': x54.X,
            'x48': x48.X,
            'x36': x36.X,
            'Objective': model.ObjVal  # Capture the objective function value
        }
        optimization_results.append(results)

# Convert results into a DataFrame
results_df = pd.DataFrame(optimization_results)

# Output results
print(results_df)

     d1   x60  x54   x48  x36     Objective
0    15   1.0 -0.0  14.0 -0.0   18553.20102
1    25   1.0 -0.0  14.0 -0.0   30922.00170
2    35   1.0 -0.0  14.0 -0.0   43290.80238
3    45   1.0 -0.0  14.0 -0.0   55659.60306
4    55   1.0 -0.0  14.0 -0.0   68028.40374
5    65   1.0 -0.0  14.0 -0.0   80397.20442
6    75   1.0 -0.0  14.0 -0.0   92766.00510
7    85   2.0  3.0  10.0 -0.0   99731.04213
8    95   3.0  7.0   5.0 -0.0  104189.60289
9   105   5.0  8.0   2.0 -0.0  108631.46826
10  115  10.0  3.0   2.0 -0.0  112322.51043
11  125  13.0  1.0   1.0 -0.0  116124.58725
12  135  13.0  0.0   1.0 -0.0  115816.47462
13  145  12.0  0.0   1.0 -0.0  115764.59424
14  155  13.0 -0.0  -0.0 -0.0  119939.44950
15  165  12.0 -0.0  -0.0 -0.0  117856.13400
16  175  10.0  1.0   0.0 -0.0  116607.73005
